<a href="https://colab.research.google.com/github/Sumaira-willis/Activity-2/blob/main/Assignment_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import random

# Sample phrases
positive_phrases = ["I love this!", "Absolutely fantastic experience.", "Couldn't be happier.", "Great job!", "This made my day."]
negative_phrases = ["I hate this.", "Terrible service.", "Very disappointed.", "Not worth it.", "Worst experience ever."]
neutral_phrases = ["It was okay.", "Nothing special.", "Average performance.", "Just another day.", "Meh."]

# Generate dataset
def generate_rows(label, phrases, count):
    return [{"id": i, "text": random.choice(phrases), "sentiment": label} for i in range(count)]

data = (
    generate_rows("positive", positive_phrases, 170) +
    generate_rows("negative", negative_phrases, 170) +
    generate_rows("neutral", neutral_phrases, 160)
)

df = pd.DataFrame(data)
df.to_csv("synthetic_sentiment_dataset.csv", index=False)


In [2]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("synthetic_sentiment_dataset.csv")

# Preview the data
print(df.head())
print(df['sentiment'].value_counts())


   id                              text sentiment
0   0                      I love this!  positive
1   1                 This made my day.  positive
2   2                      I love this!  positive
3   3  Absolutely fantastic experience.  positive
4   4                 This made my day.  positive
sentiment
positive    170
negative    170
neutral     160
Name: count, dtype: int64


In [3]:
# Preprocess the Text
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)
    text = re.sub(r"[^a-zA-Z\s]", '', text)
    text = re.sub(r"\s+", ' ', text).strip()
    stop_words = set(stopwords.words('english'))
    return ' '.join([word for word in text.split() if word not in stop_words])

df['clean_text'] = df['text'].apply(clean_text)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
# Encode Labels
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['label'] = le.fit_transform(df['sentiment'])  # positive=2, negative=0, neutral=1


In [6]:
# Vectorize Text (TF-IDF)
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=500)
X_tfidf = tfidf.fit_transform(df['clean_text'])
y = df['label']


In [7]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Vectorize
tfidf = TfidfVectorizer(max_features=500)
X_tfidf = tfidf.fit_transform(df['clean_text'])

# Convert to DataFrame for readability
tfidf_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf.get_feature_names_out())

# Print the first few rows
print(tfidf_df.head())


   absolutely  another  average  couldnt       day  disappointed  ever  \
0    0.000000      0.0      0.0      0.0  0.000000           0.0   0.0   
1    0.000000      0.0      0.0      0.0  0.634915           0.0   0.0   
2    0.000000      0.0      0.0      0.0  0.000000           0.0   0.0   
3    0.616514      0.0      0.0      0.0  0.000000           0.0   0.0   
4    0.000000      0.0      0.0      0.0  0.634915           0.0   0.0   

   experience  fantastic  great  ...      made  meh  nothing  okay  \
0    0.000000   0.000000    0.0  ...  0.000000  0.0      0.0   0.0   
1    0.000000   0.000000    0.0  ...  0.772582  0.0      0.0   0.0   
2    0.000000   0.000000    0.0  ...  0.000000  0.0      0.0   0.0   
3    0.489716   0.616514    0.0  ...  0.000000  0.0      0.0   0.0   
4    0.000000   0.000000    0.0  ...  0.772582  0.0      0.0   0.0   

   performance  service  special  terrible  worst  worth  
0          0.0      0.0      0.0       0.0    0.0    0.0  
1          0.0  

In [8]:
# Train Traditional ML Models
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Logistic Regression
lr = GridSearchCV(LogisticRegression(), {'C': [0.1, 1, 10]}, cv=5)
lr.fit(X_train, y_train)

# SVM
svm = GridSearchCV(SVC(), {'C': [0.1, 1, 10]}, cv=5)
svm.fit(X_train, y_train)


GridSearchCV(cv=5, estimator=SVC(), param_grid={'C': [0.1, 1, 10]})

In [9]:
# Evaluate Models
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# Logistic Regression
y_pred_lr = lr.predict(X_test)
print("Logistic Regression Report:")
print(classification_report(y_test, y_pred_lr))
print(confusion_matrix(y_test, y_pred_lr))

# SVM
y_pred_svm = svm.predict(X_test)
print("SVM Report:")
print(classification_report(y_test, y_pred_svm))
print(confusion_matrix(y_test, y_pred_svm))


Logistic Regression Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        27
           1       1.00      1.00      1.00        38
           2       1.00      1.00      1.00        35

    accuracy                           1.00       100
   macro avg       1.00      1.00      1.00       100
weighted avg       1.00      1.00      1.00       100

[[27  0  0]
 [ 0 38  0]
 [ 0  0 35]]
SVM Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        27
           1       1.00      1.00      1.00        38
           2       1.00      1.00      1.00        35

    accuracy                           1.00       100
   macro avg       1.00      1.00      1.00       100
weighted avg       1.00      1.00      1.00       100

[[27  0  0]
 [ 0 38  0]
 [ 0  0 35]]


In [10]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns


In [12]:
lr.fit(X_train, y_train)
svm.fit(X_train, y_train)


GridSearchCV(cv=5, estimator=SVC(), param_grid={'C': [0.1, 1, 10]})

In [13]:
# Generate predictions
y_pred_lr = lr.predict(X_test)
y_pred_svm = svm.predict(X_test)


In [14]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

# Logistic Regression Evaluation
print("Logistic Regression:")
print("Accuracy:", accuracy_score(y_test, y_pred_lr))
print("Precision:", precision_score(y_test, y_pred_lr, average='weighted'))
print("Recall:", recall_score(y_test, y_pred_lr, average='weighted'))
print("F1 Score:", f1_score(y_test, y_pred_lr, average='weighted'))
print("\nClassification Report:\n", classification_report(y_test, y_pred_lr))

# SVM Evaluation
print("\nSupport Vector Machine:")
print("Accuracy:", accuracy_score(y_test, y_pred_svm))
print("Precision:", precision_score(y_test, y_pred_svm, average='weighted'))
print("Recall:", recall_score(y_test, y_pred_svm, average='weighted'))
print("F1 Score:", f1_score(y_test, y_pred_svm, average='weighted'))
print("\nClassification Report:\n", classification_report(y_test, y_pred_svm))


Logistic Regression:
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        27
           1       1.00      1.00      1.00        38
           2       1.00      1.00      1.00        35

    accuracy                           1.00       100
   macro avg       1.00      1.00      1.00       100
weighted avg       1.00      1.00      1.00       100


Support Vector Machine:
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        27
           1       1.00      1.00      1.00        38
           2       1.00      1.00      1.00        35

    accuracy                           1.00       100
   macro avg       1.00      1.00      1.00       100
weighted avg       1.00      1.00      1.00       100

